In [0]:
employee_json="""
[
  {
    "id": 1,
    "name": "Alice",
    "age": 34,
    "address": {
      "street": "123 Maple St",
      "city": "Springfield",
      "state": "IL",
      "zip": "62701"
    },
    "phoneNumbers": [
      {
        "type": "home",
        "number": "555-1234"
      },
      {
        "type": "work",
        "number": "555-5678"
      }
    ],
    "email": "alice@example.com"
  },
  {
    "id": 2,
    "name": "Bob",
    "age": 45,
    "address": {
      "street": "456 Oak St",
      "city": "Springfield",
      "state": "IL",
      "zip": "62701"
    },
    "phoneNumbers": [
      {
        "type": "home",
        "number": "555-8765"
      },
      {
        "type": "work",
        "number": "555-4321"
      }
    ],
    "email": "bob@example.com"
  }
]
"""

In [0]:
dbutils.fs.put("/FileStore/tables/employee.json",employee_json,True)

Wrote 770 bytes.
Out[2]: True

In [0]:
df=spark.read.json('/FileStore/tables/employee.json',multiLine=True)
display(df)

address,age,email,id,name,phoneNumbers
"List(Springfield, IL, 123 Maple St, 62701)",34,alice@example.com,1,Alice,"List(List(555-1234, home), List(555-5678, work))"
"List(Springfield, IL, 456 Oak St, 62701)",45,bob@example.com,2,Bob,"List(List(555-8765, home), List(555-4321, work))"


In [0]:
from pyspark.sql.functions import *
df1=df.select("address","age","email","id","name",explode(col("phoneNumbers")).alias("mobile_number"))
display(df1)

address,age,email,id,name,mobile_number
"List(Springfield, IL, 123 Maple St, 62701)",34,alice@example.com,1,Alice,"List(555-1234, home)"
"List(Springfield, IL, 123 Maple St, 62701)",34,alice@example.com,1,Alice,"List(555-5678, work)"
"List(Springfield, IL, 456 Oak St, 62701)",45,bob@example.com,2,Bob,"List(555-8765, home)"
"List(Springfield, IL, 456 Oak St, 62701)",45,bob@example.com,2,Bob,"List(555-4321, work)"


In [0]:
#conver the struct type into json type then only with json tuple we can split the columns
df_json=df1.select(to_json(col("address")).alias("address"),"age","email","id","name",to_json(col("mobile_number")).alias("mobile_number"))


In [0]:
#json tuple to seperate the json nested type into seperate columns and The json_tuple function in Spark can be used to extract multiple fields from a JSON column into separate columns. 
df_json_tuple_1=df_json.select("address","age","email","id","name",json_tuple(col("mobile_number"),"number","type").alias("number","type"))
df_json_tuple_2=df_json_tuple_1.select(json_tuple(col("address"),"city","state","street","zip").alias("city","state","street","zip"),"age","email","id","name","number","type")
display(df_json_tuple_2)


city,state,street,zip,age,email,id,name,number,type
Springfield,IL,123 Maple St,62701,34,alice@example.com,1,Alice,555-1234,home
Springfield,IL,123 Maple St,62701,34,alice@example.com,1,Alice,555-5678,work
Springfield,IL,456 Oak St,62701,45,bob@example.com,2,Bob,555-8765,home
Springfield,IL,456 Oak St,62701,45,bob@example.com,2,Bob,555-4321,work
